In [ ]:

!pip install rotary-embedding-torch
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [ ]:
!pip install pytorch-ignite
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tiktoken
import torch
from torch.nn import CrossEntropyLoss
from torchtext.data.metrics import bleu_score
from ignite.metrics import Rouge
from transformers import AdamW

from datetime import datetime
import os
import random

import gc
import wandb
import warnings
import math
import time
import math
import pickle
from typing import Tuple
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import inspect
from rotary_embedding_torch import RotaryEmbedding


In [ ]:
enc = tiktoken.get_encoding("gpt2")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.empty_cache()
gc.collect()

# Ignore all warnings
warnings.filterwarnings("ignore")
SUMMARY_ROOT = '/content/drive/MyDrive/HW5-Data&Model/DailyMail_Summary'
SQUAD_ROOT = '/content/drive/MyDrive/HW5-Data&Model/QA-Dataset'
SENTIMENT_ROOT = '/content/drive/MyDrive/HW5-Data&Model/SentimentAnalysis-Dataset'
NER_ROOT = '/content/drive/MyDrive/HW5-Data&Model/NER_Dataset'

QA_Index = 0
Sentiment_Index= 0
## Dataset Class
class MergedDataset(Dataset):
    def __init__(self, summary_root, squad_root,sentiment_root, ner_root, file, length=None):
        # Merge the datasets for the summarizer and QA tasks
        self.summarise_data = np.load(os.path.join(summary_root, file+'.npy'), mmap_mode='r')[:length]
        self.summarise_lens = np.load(os.path.join(summary_root, file+'_lens.npy'), mmap_mode='r')[:length]

        self.qa_data = np.load(os.path.join(squad_root, file+'.npy'), mmap_mode='r')[:length]
        self.qa_lens = np.load(os.path.join(squad_root, file+'_lens.npy'), mmap_mode='r')[:length]

        self.sentiment_data = np.load(os.path.join(sentiment_root, file+'.npy'), mmap_mode='r')[:length]
        self.sentiment_lens = np.load(os.path.join(sentiment_root, file+'_lens.npy'), mmap_mode='r')[:length]

        self.ner_data = np.load(os.path.join(ner_root, file+'.npy'), mmap_mode='r')[:length]
        self.ner_lens = np.load(os.path.join(ner_root, file+'_lens.npy'), mmap_mode='r')[:length]

        self.data = np.concatenate([self.summarise_data, self.qa_data, self.sentiment_data, self.ner_data])
        self.data_lens = np.concatenate([self.summarise_lens, self.qa_lens, self.sentiment_lens, self.ner_lens])

        self.length = self.data.shape[0]

        QA_Index = len(self.summarise_data)+len(self.qa_data)
        Sentiment_Index = len(self.summarise_data) + len(self.qa_data) + len(self.sentiment_data)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        d = self.data[idx]
        l = self.data_lens[idx]

#         if idx<len(self.summarise_data):
#           # Summary Data
#             dataSetTag = 0
#         elif ((idx>len(self.summarise_data)) and (idx<(len(self.summarise_data)+len(self.qa_data)))):
#           # QA data
#             dataSetTag = 1
#         elif ((idx>(len(self.summarise_data)+len(self.qa_data))) and (idx<(len(self.summarise_data)+len(self.qa_data)+len(self.sentiment_data)))):
#           #Sentiment Data
#             dataSetTag = 2
#         elif (idx>(len(self.summarise_data)+len(self.qa_data)+len(self.sentiment_data))) and (idx<(len(self.summarise_data)+len(self.qa_data)+len(self.sentiment_data)+len(self.ner_data))):
#           #NER Data
#             dataSetTag = 3

        if idx < len(self.summarise_data):
        # Summary Data
            dataSetTag = 0
        elif idx < (len(self.summarise_data) + len(self.qa_data)):
            # QA data
            dataSetTag = 1
        elif idx < (len(self.summarise_data) + len(self.qa_data) + len(self.sentiment_data)):
            # Sentiment Data
            dataSetTag = 2
        elif idx < (len(self.summarise_data) + len(self.qa_data) + len(self.sentiment_data) + len(self.ner_data)):
            # NER Data
            dataSetTag = 3
        else:
            print("Error in get item index\n")

        return d, l, dataSetTag

In [ ]:
class ModelConfig:
  batch_size: int = 2
  block_size: int = 1024
  vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency

  dim: int = 512
  n_layers: int = 8
  n_heads: int = 8
  max_seq_len: int = 512
  layer_norm_eps: float = 1e-6
  dropout: float = 0.0
  hidden_dim: int = None
  n_embd: int = 1024
  multiple_of: int = 32
  rope_dim: int = 64
  bias: bool = True

  weight_decay = 1e-1
  betas = (0.9, 0.99)
  eval_iters = 50
  master_process = True
  warmup_iters = 10
  learning_rate = 0.001
  lr_decay_iters = 150
  min_lr = 6e-5
  wandb_log = True
  wandb_project = 'HW5'
  wandb_run_name = 'Pretrain_29_4_v1'
  decay_lr = True
  eval_interval = 100
  eval_only = False
  grad_clip = 1.0
  max_iters = 5000
  gradient_accumulation_steps = 2
  save_checkpoint_iters = 100
  log_interval = 5
config = ModelConfig()

In [ ]:
train_dataset = MergedDataset(SUMMARY_ROOT, SQUAD_ROOT, SENTIMENT_ROOT, NER_ROOT, 'train', length=75000)
val_dataset = MergedDataset(SUMMARY_ROOT, SQUAD_ROOT, SENTIMENT_ROOT, NER_ROOT, 'validation', length=300)

## Intiialize dataloader
train_dataloader =  DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=4)

val_dataloader =  DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=4)

In [ ]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        """
        Initialize the RMSNorm normalization layer.
        Args:
            dim (int): The dimension of the input tensor.
            eps (float, optional): A small value added to the denominator for numerical stability. Default is 1e-6.
        Attributes:
            eps (float): A small value added to the denominator for numerical stability.
            weight (nn.Parameter): Learnable scaling parameter.
        """
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        """
        Apply the RMSNorm normalization to the input tensor.
        Args:
            x (torch.Tensor): The input tensor.
        Returns:
            torch.Tensor: The normalized tensor.
        """
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        """
        Forward pass through the RMSNorm layer.
        Args:
            x (torch.Tensor): The input tensor.
        Returns:
            torch.Tensor: The output tensor after applying RMSNorm.
        """
        output = self._norm(x.float()).type_as(x)
        return output * self.weight

class FeedForward(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, multiple_of: int, dropout: float):
        super().__init__()
        if hidden_dim is None:
            hidden_dim = 4 * dim
            hidden_dim = int(2 * hidden_dim / 3)
            hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        self.w1 = nn.Linear(dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.w3 = nn.Linear(dim, hidden_dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def SwiGLU(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Compute the SwiGLU activation function (see Section 2 in
        https://arxiv.org/abs/2204.02311
        '''
        return F.silu(self.w1(x)) * self.w3(x)

    def forward(self, x):
        return self.dropout(self.w2(self.SwiGLU(x)))


class CausalSelfAttention(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        assert config.n_embd % config.n_heads == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_heads
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        self.rotary = RotaryEmbedding(config.rope_dim)
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # apply RoPE, see https://arxiv.org/abs/2104.09864
        k = self.rotary.rotate_queries_or_keys(k)
        q = self.rotary.rotate_queries_or_keys(q)


        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.rn_1 = RMSNorm(config.n_embd, eps=config.layer_norm_eps)
        self.attn = CausalSelfAttention(config)
        self.rn_2 = RMSNorm(config.n_embd, eps=config.layer_norm_eps)
        self.mlp = FeedForward(config.n_embd, config.hidden_dim, config.multiple_of, config.dropout)

    def forward(self, x):
        x = x + self.attn(self.rn_1(x))
        x = x + self.mlp(self.rn_2(x))
        return x

class GPT(nn.Module):

    def __init__(self, config: ModelConfig):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layers)]),
            ln_f = RMSNorm(config.n_embd, eps=config.layer_norm_eps)
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layers))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, return_all_logits=False):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        x = self.transformer.drop(tok_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
          if return_all_logits:
            logits = self.lm_head(x)
            # inference-time mini-optimization: only forward the lm_head on the very last position
          else:
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim

          loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -config.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = GPT(config)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

152.749312 M parameters


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
## Load pretrained model ##
# CHK_PT_PATH = '/content/drive/MyDrive/HW5-Data&Model/checkpoint_iter_1000.pth'
# checkpoint = torch.load(CHK_PT_PATH, map_location=device)

checkpoint = torch.load("/content/drive/MyDrive/HW5-Data&Model/checkpoint_iter_1000.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# checkpoint_model_args = checkpoint['model_args']
# checkpoint_model_args['dropout'] = dropout

# gptconf = GPTConfig(**checkpoint_model_args)

# state_dict = checkpoint['model']

# unwanted_prefix = '_orig_mod.'
# for k,v in list(state_dict.items()):
#     if k.startswith(unwanted_prefix):
#         state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

# model.load_state_dict(state_dict)

# iter_num = checkpoint['iter_num']
# best_val_loss = checkpoint['best_val_loss']

iter_num = 0
best_val_loss = 20
# Setup training functions
IGNORE_INDEX = -1
loss_fct = CrossEntropyLoss(ignore_index=IGNORE_INDEX)
val_loss_fct = CrossEntropyLoss(ignore_index=IGNORE_INDEX)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, betas=config.betas)
scaler = torch.cuda.amp.GradScaler(enabled=True)

In [ ]:
## Init wandb
wandb.login(key="197d96ebfe1ad37dfd2180d901ca0f779e76bdfe")
wandb_config = {
    'BATCH_SIZE': config.batch_size,
    'learning_rate': config.learning_rate,
    'gradient_accumulation_steps': config.gradient_accumulation_steps
}
wandb_run_name = 'FineTunev1'
wandb.init(project=config.wandb_project, name=wandb_run_name, config=wandb_config)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abasrith. Use `wandb login --relogin` to force relogin


In [ ]:
model = model.to(device)

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < config.warmup_iters:
        return config.learning_rate * it / config.warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > config.lr_decay_iters:
        return config.min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - config.warmup_iters) / (config.lr_decay_iters - config.warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return config.min_lr + coeff * (config.learning_rate - config.min_lr)

In [ ]:
def train(model):
    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    set_seed(1337)
    EPOCHS = 2
    for _ in np.arange(EPOCHS):
        for step, (data, article_len, datasetTag) in enumerate(train_dataloader):
            inputs, labels = torch.tensor(data), torch.tensor(data)
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.train()
            logits = model(inputs, return_all_logits=True)[0]

            # only consider loss on reference summary just like seq2seq models
            shift_logits = []
            shift_labels = []
            for batch_idx in range(logits.shape[0]):
                idx = article_len[batch_idx].item() # index of separator token
                shift_logits.append(logits[batch_idx, idx:-1, :])
                shift_labels.append(labels[batch_idx, idx+1:])
            shift_logits = torch.cat(shift_logits, dim=0)
            shift_labels = torch.cat(shift_labels, dim=0)

            loss = loss_fct(shift_logits, shift_labels)
            loss = loss/config.gradient_accumulation_steps
            scaler.scale(loss).backward()

            tr_loss += loss.item()
            if (step + 1) % config.gradient_accumulation_steps == 0:
                lr = get_lr(step)
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr

                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)

                scaler.step(optimizer)
                scaler.update()

                model.zero_grad()
                optimizer.zero_grad(set_to_none=True)

                global_step += 1
                logging_loss = tr_loss
                print("loss:", loss.item(), end='\n\n')

                if (step + 1)/config.gradient_accumulation_steps == 1.0:
                    print('After 1st update: ', end='\n\n')
                    generate_sample(1) #Summary
                    generate_sample(QA_Index+1) #QA
                    generate_sample(Sentiment_Index+1) #Sentiment
                    generate_sample(-5) #NER


            if (step + 1) % (20*config.gradient_accumulation_steps) == 0:
                results = evaluate(model, global_step, lr, loss.item())
                print('After', global_step+1,'updates: ', end='\n\n')
                generate_sample(1) #Summary
                generate_sample(QA_Index+1) #QA
                generate_sample(Sentiment_Index+1) #Sentiment
                generate_sample(-5) #NER


            del inputs, labels
            torch.cuda.empty_cache()
            gc.collect()

In [ ]:
def generate_sample(index):
    data_sample, art_len_sample, _ = val_dataset[index]
    data_sample = torch.tensor(data_sample[None,:]).to(device)
    idx = art_len_sample.item()

    logits = model(data_sample, return_all_logits=True)[0]
    preds = logits[0, idx:-1, :].argmax(dim=-1).tolist()

    labels = data_sample[0, idx+1:].tolist()

    if index == 0:
        print("Pred Summary:\n %s \n" % enc.decode(preds))
        print("True Summary:\n %s \n\n" % enc.decode(labels))
    elif index == QA_Index:
        print("Pred Answer:\n %s \n" % enc.decode(preds))
        print("True Answer:\n %s \n\n" % enc.decode(labels))
    elif index == Sentiment_Index:
        print("Pred Sentiment:\n %s \n" % enc.decode(preds))
        print("True Sentiment:\n %s \n\n" % enc.decode(labels))
    else:
        print("Pred NER:\n %s \n" % enc.decode(preds))
        print("True NER:\n %s \n\n" % enc.decode(labels))

In [ ]:
OUTPUT_DIR = '/content/'
def evaluate(model, global_step=None, lr=None, tr_loss=None):
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    eval_output_dir = OUTPUT_DIR

    results = {}

    eval_loss = 0.0
    eval_bleu_scores = 0.0
    eval_rouge_scores = 0.0
    nb_eval_steps = 0
    model.eval()

    for (data, article_len, datasetTag) in val_dataloader:
        inputs, labels = torch.tensor(data).to(device), torch.tensor(data).to(device)
        with torch.no_grad():
            logits = model(inputs, return_all_logits=True)[0]
            shift_logits = []
            shift_labels = []
            avg_eval_bleu = 0.0
            avg_rouge_score = 0.0
            m = Rouge(variants=["L",1,2], multiref="best")

            for batch_idx in range(logits.shape[0]):
                idx = article_len[batch_idx].item() # index of separator token

                shift_logits.append(logits[batch_idx, idx:-1, :])
                shift_labels.append(labels[batch_idx, idx+1:])

                greedy_labels = labels[batch_idx, idx+1:].tolist()
                index = greedy_labels.index(enc.eot_token)
                greedy_labels  = greedy_labels[:index]
                references = [[enc.decode(greedy_labels).split()]]

                greedy_preds = logits[batch_idx, idx:-1, :].argmax(dim=-1).tolist()
                greedy_preds = greedy_preds[:index]
                hypotheses = [enc.decode(greedy_preds).split()]

                if datasetTag[batch_idx].item() == 0:
                    bleu4 = bleu_score(hypotheses, references, max_n=2, weights=[0.5, 0.5])
                    avg_eval_bleu += bleu4
                elif datasetTag[batch_idx].item() == 1:
                    m.update((hypotheses, references))
                    rouge = m.compute()
                    avg_rouge_score += max(rouge.values())
                elif datasetTag[batch_idx].item() == 2:
                    print(" Here at Sentiment Eval\n")
                elif datasetTag[batch_idx].item() == 3:
                    print(" Here at NER\n")
                else:
                    print(" Error Batch")


            shift_logits = torch.cat(shift_logits, dim=0)
            shift_labels = torch.cat(shift_labels, dim=0)

            lm_loss = loss_fct(shift_logits, shift_labels)
            eval_loss += lm_loss.mean().item()

            eval_bleu_scores += avg_eval_bleu/logits.shape[0]
            eval_rouge_scores += avg_rouge_score/logits.shape[0]

        del inputs, labels
        torch.cuda.empty_cache()
        gc.collect()


        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_bleu_scores = 2*eval_bleu_scores / nb_eval_steps
    eval_rouge_scores = 2*eval_rouge_scores / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {
        "perplexity": perplexity,
        'eval_bleu_scores': eval_bleu_scores,
        'eval_rouge_scores': eval_rouge_scores
    }
    print("perplexity:", perplexity.item())
    print('eval_bleu_scores: ', eval_bleu_scores)
    print('eval_rouge_scores: ', eval_rouge_scores)

    global best_bleu_score
    global best_rouge_score
    if global_step:
        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "a") as f:
            for key in sorted(result.keys()):
                f.write('\n\n')
                f.write("time = %s, %s = %s, step = %s\n" % (datetime.now().strftime("%d/%m/%Y %H:%M:%S"), key, str(result[key]), str(global_step)))

        wandb.log({
                "iter": global_step,
                "train/loss": tr_loss,
                "val/loss": eval_loss,
                'eval_bleu_scores': eval_bleu_scores,
                'eval_rouge_scores': eval_rouge_scores,
                "lr": lr,
            })

        if eval_bleu_scores >= best_bleu_score:
            best_bleu_score = eval_bleu_scores
            # checkpoint = {
            #     'model': model.state_dict(),
            #     'optimizer': optimizer.state_dict(),
            #     'model_args': checkpoint_model_args,
            #     'iter_num': global_step,
            #     'best_val_loss': min(best_val_loss, eval_loss),
            #     'best_bleu_score': best_bleu_score,
            #     'best_rouge_score': best_rouge_score,
            #     'config': gptconf,
            # }
            checkpoint = {
                'model': model.state_dict()
            }
            print(f"saving checkpoint to {eval_output_dir}")
            torch.save(checkpoint, os.path.join(eval_output_dir, 'bleu_ckpt.pt'))

        if eval_rouge_scores >= best_rouge_score:
            best_rouge_score = eval_rouge_scores
            checkpoint = {
                'model': model.state_dict()
            }
            print(f"saving checkpoint to {eval_output_dir}")
            torch.save(checkpoint, os.path.join(eval_output_dir, 'rouge_ckpt.pt'))

    return result

In [ ]:
best_bleu_score = 0.21
best_rouge_score = 0.43

train(model)

loss: 5.115752220153809

After 1st update: 

Pred NER:
 .,,,,.,,, the., the, the,

 the was said the first of. the a interview..

.,,,. he was a3- to, to
 first, to of of the,,., in the House,,
TheTheTheAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATheAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATheAAAAAAAAAAAAAAAAATheAAAAATheTheAAAAATheAAAAAAAAAAAATheAAAATheTheAAAAAAAAAAATheAAAAAAAAAATheATheAAATheATheTheAAATheTheAAAAAAAAAATheTheTheATheATheTheTheTheATheAAAAAATheTheTheTheTheTheAATheTheTheTheTheTheTheTheTheTheTheATheTheTheTheAATheTheAATheAAATheTheAATheAATheTheTheTheTheTheTheAAAAATheATheAAATheTheTheTheTheTheTheTheTheAAAAAAAAAATheTheATheTheATheTheTheTheTheTheTheTheTheTheTheATheTheAAATheATheATheTheTheTheTheTheTheTheTheTheATheTheTheTheTheTheTheTheTheTheTheTheTheAAAATheTheTheTheTheTheTheTheTheTheTheThe 

True NER:
 afetimbi Gomis collapses within 10 minutes of kickoff at Tottenham .
But he reportedly left the pitch conscious and weari